### Imports

In [2]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps.storymap import JournalStoryMap
from arcgis.mapping import WebMap
from arcgis.features import FeatureLayer, Feature
from arcgis.geometry import Geometry, SpatialReference, Point

### Utility Methods

In [57]:
import uuid,json,arcpy
    
def getExtent(fromFeature):
    g = Geometry(fromFeature.geometry)
    if g.type.lower() == 'point':
        bg = Point({"x": g.x, "y": g.y, "spatialReference": {"wkid": 102100}})
        gbuf = bg.buffer(1609.344) # assume meters, 1609.344 ~ 1 mile
        return gbuf.extent
    
    return g.extent

def getMapAction(fromFeature, forWebmap, layer, label_field="objectid", label_prefix="Feature Extent", add_popup=False):
    # TODO - make sure input parameters are valid, since we are just inserting them into the mapaction schema
    
    # pull out a value from the feature to display in our anchor tag
    lbl_val = fromFeature.attributes[fromFeature.fields[0]]
    if label_field in fromFeature.fields:
        lbl_val = fromFeature.attributes[label_field]
    
    # format the label
    lbl_frmtd = '{0}{1}'.format(label_prefix, lbl_val)
    
    # get the geometry extent
    # TODO - Need to validate this or default to a valid extent
    
    ext = getExtent(fromFeature)

    
    popup_def = None
    if add_popup:
        popup_def = {
            "layerId": layer.id,
            "fieldName": "OBJECTID",
            "fieldValue": fromFeature.attributes['OBJECTID'],
            "anchorPoint": {
                "x": Geometry(fromFeature.geometry).centroid[0],
                "y": Geometry(fromFeature.geometry).centroid[1],
                "spatialReference": {
                    "wkid": 102100
                }
            }
        }
    
    print('Popup: {0}'.format(popup_def))
    
    # create a unique id
    # ts = int(round(time.time() * 1000)) this isn't precise enough, gets duplicated in loop
    ts = str(uuid.uuid4())
    #print('id : {0}'.format(ts))
    
    # insert values into our map action schema
    ma = {
        "id":"MJ-ACTION-{0}".format(ts),
        "type": "media",
        "media": {
            "type":"webmap",
            "webmap": {
                "id":forWebmap.id,
                "extent": {
                    "xmin": ext[0],
                    "ymin": ext[1],
                    "xmax": ext[2],
                    "ymax": ext[3],
                    "spatialReference":{
                        "wkid": 102100
                    }
                },
                "layers": None,
                "popup": popup_def,
                "overview":{
                    "enable": False,
                    "openByDefault": False
                },
                "legend": {
                    "enable": False,
                    "openByDefault": False
                },
                "geocoder":{
                    "enable": False
                },
                "altText": ""
            }
        }
    }
    
    # create the html string
    content_anchor = '<p><a data-storymaps="{0}" data-storymaps-type="media">{1}</a></p><p>&nbsp;</p>'.format(ma['id'], lbl_frmtd)
    
    # hand back our results to whoever called this function
    return {"content_link": content_anchor, "map_action": ma}

### Connect to Portal

In [7]:
prtl = GIS(username='marktorrey')

Enter password: ········


### Get a webmap that has some data we want to query

In [42]:
wm_itm = prtl.content.get('1ad33784a4824ef38f2ce1c60b2963e0')

wm = WebMap(wm_itm)
for l in wm.layers:
    print(l)
point_layer = FeatureLayer(wm.layers[0].url)
poly_layer = FeatureLayer(wm.layers[2].url)

{
  "id": "hosted_point_layer_8467",
  "layerType": "ArcGISFeatureLayer",
  "url": "https://services.arcgis.com/dfRzIozUOVYoi65F/arcgis/rest/services/hosted_point_layer/FeatureServer/0",
  "visibility": true,
  "opacity": 1,
  "title": "hosted_point_layer",
  "itemId": "e20b714c7f8640c3a50a3636270741c7",
  "popupInfo": {
    "title": "hosted_point_layer",
    "fieldInfos": [
      {
        "fieldName": "OBJECTID",
        "label": "OBJECTID",
        "isEditable": false,
        "tooltip": "",
        "visible": false,
        "stringFieldOption": "textbox"
      },
      {
        "fieldName": "textfield",
        "label": "Text Field",
        "isEditable": true,
        "tooltip": "",
        "visible": true,
        "stringFieldOption": "textbox"
      },
      {
        "fieldName": "integerfield",
        "label": "Integer Field",
        "isEditable": true,
        "tooltip": "",
        "visible": true,
        "stringFieldOption": "textbox",
        "format": {
          "pla

### Get the story map

In [37]:
itm = prtl.content.get('876f3dc003184f2ea1d2785e294fd983')
sm = JournalStoryMap(itm)
sections = sm.properties['values']['story']['sections']

### Clear out our test section

In [55]:
# clear out any stuff we tried previously
sections[1]['content'] = ' ' 
sections[1]['contentActions'] = []
sm.save()

True

### Iterate over our features and generate the map actions

In [58]:
poly_features = poly_layer.query(out_fields='*')
point_features = point_layer.query(out_fields='*')
for f in point_features.features:
    print(f)
    ma = getMapAction(f, wm_itm, wm.layers[0], label_field='textfield',add_popup=True)
    sections[1]['content'] += ma['content_link']
    sections[1]['contentActions'].append(ma['map_action'])
    
sm.save()


{"geometry": {"x": -8959336.823473882, "y": 4226616.209377475}, "attributes": {"OBJECTID": 1, "textfield": "Charlotte", "integerfield": null, "doublefield": null, "datefield": null, "codeddomain": null, "rangedomain": null, "GlobalID": "7e537c0a-205c-49a0-a39e-15255e71025f", "CreationDate": 1533060370081, "Creator": "marktorrey", "EditDate": 1535477743983, "Editor": "marktorrey"}}
Popup: {'layerId': 'hosted_point_layer_8467', 'fieldName': 'OBJECTID', 'fieldValue': 1, 'anchorPoint': {'x': -8959336.823473882, 'y': 4226616.209377475, 'spatialReference': {'wkid': 102100}}}
{"geometry": {"x": -9978089.536458423, "y": 4833253.777132536}, "attributes": {"OBJECTID": 2, "textfield": "Springfield", "integerfield": null, "doublefield": null, "datefield": null, "codeddomain": null, "rangedomain": null, "GlobalID": "caa40861-4015-468b-98f9-16fe90bee4a2", "CreationDate": 1535477696994, "Creator": "marktorrey", "EditDate": 1535477705465, "Editor": "marktorrey"}}
Popup: {'layerId': 'hosted_point_layer

True